In [1]:
import json
import pandas as pd

In [29]:
with open('data/p0.json', 'r') as f:
    df0 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

with open('data/p1.json', 'r') as f:
    df1 = pd.DataFrame.from_dict(json.load(f)['leaderboard'], orient='columns')

df = pd.concat([df0, df1])
df = df.drop(['userId', 'name', 'avatar', 'addresses', 'twitterId', 'discord', 'twitter', 'discordId', 'connectedWallet'], axis=1)
len(df)

1820

In [34]:
round_svy_total = 6000
total_xp = df['xp'].sum()
total_xp

3356250

In [35]:
df['proportion'] = df['xp'] / total_xp
df['amount'] = df['proportion'] * round_svy_total

In [36]:
df

,xp,numberOfQuests,address,proportion,amount
0,14995,141,0x51F6749Dee3510F983C15B7c239CC6df4e4054Ea,0.004468,26.806704
1,14530,119,0x7c318bfBe63833E7a291e130337652C94C6EE464,0.004329,25.975419
2,12030,87,0x46f3cC6a1c00A5cD8864d2B92f128196CAE07D15,0.003584,21.506145
3,11470,97,0x56EbC608fD6B8F8CeF2775a3Ee0DddaEc01A7000,0.003418,20.505028
4,11240,96,0xa3732fC9d975f9Aa1cfD069C991cDd3714d7c8f8,0.003349,20.093855
...,...,...,...,...,...
815,10,1,0xEc263df053412D5816bd1B5BF3736bAf2809163a,0.000003,0.017877
816,10,1,0x3DA972d8F5bD733A59f1c77cCc1953dAc2862343,0.000003,0.017877
817,10,1,0xe0854F6B6c06587Ac3f1f4dbC992858bf8725224,0.000003,0.017877
818,10,1,0x8FD99800307a2a761403441E634d97f191BCe45D,0.000003,0.017877


In [33]:
df.to_csv('data/round1.csv')